In [ ]:
from ray import tune
import ray
import copy
import yaml
import train_and_eval
import glob
import json
ray.init(log_to_driver=False)

In [ ]:
def config_helper(config):
    return train_and_eval.run_experiment(**config)

In [ ]:
with open('configs/gpn/ood_loc_gpn_16.yaml') as f:
    config = yaml.safe_load(f)

In [ ]:
# dirs = sorted(glob.glob("../ray_results/ray_helper_2022*"))
# trials = glob.glob(dirs[-1] + "/*")

In [ ]:
# rocs = []
# for a_trial in trials:
#     if ".json" in a_trial:
#         continue
#     with open(a_trial + "/result.json", 'r') as f:
#         try:
#             a_result_file = json.load(f)
#         except:
#             continue
#     rocs.append(a_result_file['test_ood_detection_epistemic_auroc'])
# max(rocs)

In [ ]:
datasets = ['CoraML', 'CiteSeer', 'PubMed', 'AmazonPhotos', 'AmazonComputers', 'CoauthorCS', 'CoauthorPhysics']

In [ ]:
tune.retry()

In [133]:
config['model']['latent_dist_reg'] = tune.grid_search([0])#[10 ** i for i in range(-6, -2)])
config['model']['KNN_K'] = 0 #tune.grid_search([1, 2, 3])
config['model']['normalize_dist_reg'] = 0 #tune.grid_search([False, ])
config['model']['dist_sigma'] = 0 #tune.grid_search([10 ** i for i in range(-1, 3)])
config['model']['dist_embedding_beta'] = False #tune.grid_search([True, False])
config['data']['dataset'] = tune.grid_search(datasets)
results = tune.run(
    config_helper,
    metric='test_ood_detection_epistemic_auroc',
    mode="max", 
    num_samples=1,
    max_concurrent_trials=8,
    resources_per_trial={"gpu": 1, 'cpu': 8},
    config=config,
    max_failures=2)

Trial name,status,loc,data/dataset,model/latent_dist...,iter,total time (s),test_accuracy,test_brier_score,test_ECE
config_helper_3357f_00000,TERMINATED,10.176.130.19:702596,CoraML,0,1,14.0213,0.882739,0.269894,0.0616859
config_helper_3357f_00001,TERMINATED,10.176.130.19:702663,CiteSeer,0,1,10.0308,0.709812,0.513235,0.020887
config_helper_3357f_00002,TERMINATED,10.176.130.19:702726,PubMed,0,1,17.3406,0.940923,0.172281,0.0404932
config_helper_3357f_00003,TERMINATED,10.176.130.19:702789,AmazonPhotos,0,1,18.1516,0.936481,0.271825,0.135985
config_helper_3357f_00004,TERMINATED,10.176.130.19:702851,AmazonComputers,0,1,30.2803,0.863302,0.323405,0.0858122
config_helper_3357f_00005,TERMINATED,10.176.130.19:702914,CoauthorCS,0,1,37.7249,0.873292,0.370981,0.170008
config_helper_3357f_00006,TERMINATED,10.176.130.19:702977,CoauthorPhysics,0,1,31.3256,0.974824,0.143677,0.0779758


Result for config_helper_3357f_00002:
  date: 2022-10-27_16-56-59
  done: false
  experiment_id: 2aba10e6f65344f1816b725ae550cfe5
  hostname: csr-93141
  iterations_since_restore: 1
  node_ip: 10.176.130.19
  pid: 702726
  test_CE: 0.16932712495326996
  test_ECE: 0.0404932014644146
  test_accuracy: 0.940922737121582
  test_average_entropy: 0.25843626260757446
  test_avg_prediction_confidence_aleatoric: 0.9004295468330383
  test_avg_prediction_confidence_epistemic: 37781.37890625
  test_avg_sample_confidence_aleatoric: 0.9004295468330383
  test_avg_sample_confidence_epistemic: 41262.26171875
  test_avg_sample_confidence_features: 50972.9609375
  test_avg_sample_confidence_neighborhood: .nan
  test_brier_score: 0.17228084802627563
  test_confidence_aleatoric_apr: 0.9933706312329983
  test_confidence_aleatoric_auroc: 0.9060270784904982
  test_confidence_epistemic_apr: 0.9870254057928272
  test_confidence_epistemic_auroc: 0.8568993727004857
  test_confidence_structure_apr: .nan
  test_conf

2022-10-27 16:57:38,509	INFO tune.py:758 -- Total run time: 73.76 seconds (73.53 seconds for the tuning loop).


In [134]:
results.results_df[['config/data/dataset', 'test_ood_detection_epistemic_auroc']]

,config/data/dataset,test_ood_detection_epistemic_auroc
trial_id,,
3357f_00000,CoraML,0.852011
3357f_00001,CiteSeer,0.737388
3357f_00002,PubMed,0.692579
3357f_00003,AmazonPhotos,0.877661
3357f_00004,AmazonComputers,0.876548
3357f_00005,CoauthorCS,0.930296
3357f_00006,CoauthorPhysics,0.909575


In [ ]:
results_new = results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df_old = df

In [ ]:
results.results_df

In [ ]:
df

In [ ]:
param_keys = ['latent_dist_reg', 'KNN_K', 'normalize_dist_reg', 'dist_sigma', 'dist_embedding_beta']#, 'config/data/dataset',
result_keys = ['val_accuracy', 'test_ood_detection_epistemic_auroc', 'test_accuracy']


In [ ]:
recent_dirs = sorted(glob.glob("../ray_results/*"))[-2:]
params = []
results = []
for a_recent_dir in recent_dirs:
    params.extend(sorted(glob.glob(a_recent_dir+"/*/params.json")))
    results.extend(sorted(glob.glob(a_recent_dir+"/*/result.json")))

In [ ]:
len(results)

In [131]:
dicts = []
for (a_param, a_result) in zip(params, results):
    with open(a_param) as f:
        a_param_dict = yaml.safe_load(f)
    param_dict = {a_key: a_param_dict['model'][a_key] for a_key in param_keys}
    with open(a_result) as f:
        a_result_dict = yaml.safe_load(f)
    # print(a_result_dict.keys())
    try: 
        result_dict = {a_key: a_result_dict[a_key]  for a_key in result_keys}
    except:
        result_dict = dict()
    dicts.append(param_dict | result_dict)

In [132]:
df = pd.DataFrame.from_dicts(dicts)

AttributeError: type object 'DataFrame' has no attribute 'from_dicts'

In [ ]:
# dfs = {}
for a_dataset in pd.unique(df["config/data/dataset"]):
    dfs[a_dataset] = df[df["config/data/dataset"] == a_dataset]

In [ ]:
a_dataset = datasets[7]
dfs[a_dataset]['training_iteration']
# for a_key in dfs[a_dataset].keys():
#     print(a_key)
all_keys = ["config/model/" + a_key for a_key in keys] + ['config/data/dataset', 'val_accuracy', 'test_ood_detection_epistemic_auroc', 'test_accuracy']
dfs[a_dataset][all_keys].sort_values('val_accuracy')

In [ ]:
for a_dataset in datasets:
    curr_df =  dfs[a_dataset][all_keys]
    print(a_dataset)
    for i, KNN_K in enumerate(pd.unique(curr_df['config/model/KNN_K'])):
        K_df = curr_df[curr_df['config/model/KNN_K'] == KNN_K]
        val_selected_test_acc = K_df.iloc[np.argmax(K_df['val_accuracy'])]['test_accuracy']
        val_selected_roc = K_df.iloc[np.argmax(K_df['val_accuracy'])]['test_ood_detection_epistemic_auroc']
        best_roc = K_df.iloc[np.argmax(K_df['test_ood_detection_epistemic_auroc'])]['test_ood_detection_epistemic_auroc']
        print(f"for k={KNN_K} best val selected accuracy {val_selected_test_acc:.4f}, selected ROC: {val_selected_roc:.4f}, best overall {best_roc:.4f}")
        # curr_df[curr_df['config/model/KNN_K'] == KNN_K].plot.scatter(x='config/model/latent_dist_reg', y='config/model/dist_sigma', c='test_ood_detection_epistemic_auroc', logx=True, logy=True, vmin=.8628)
        # plt.title(f"KNN_K = {KNN_K}")
        # plt.show()
    print()